In [90]:
import torch
import torch.optim as optim 
import torch.nn as nn

In [16]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0] #known data
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4] #unknown data
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [17]:
def model(t_u, w, b):
    return w * t_u + b

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [18]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None: 
            params.grad.zero_() # 这可以在调用backward之前在循环中的任何时候完成
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        loss.backward()
        params = (params - learning_rate * params.grad).detach().requires_grad_()
        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
    return params

In [19]:
t_un = 0.1 * t_u
training_loop(
    n_epochs = 5000,
    learning_rate = 1e-2,
    params = torch.tensor([1.0, 0.0], requires_grad=True),
    t_u = t_un,
    t_c = t_c)

Epoch 500, Loss 7.860115
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957698
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)

In [32]:
params = torch.tensor([1.0, 0.0], requires_grad = True)
learning_rate = 1e-4
optimizer = optim.SGD([params], lr = learning_rate)

In [25]:
loss = loss_fn(model(t_u, *params), t_c)
loss.backward()
optimizer.step()
params

tensor([ 0.0965, -0.0165], requires_grad=True)

In [28]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    optimizer = optim.SGD([params], lr = learning_rate)
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None: 
            params.grad.zero_() # 这可以在调用backward之前在循环中的任何时候完成
        loss = loss_fn(model(t_u, *params), t_c)
        loss.backward()
        optimizer.step()
        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
    return params

In [35]:
def training_loop(n_epochs, optimizer, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        loss = loss_fn(model(t_u, *params), t_c)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
    return params

In [38]:
params = torch.tensor([1.0, 0.0], requires_grad = True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr = learning_rate)

In [39]:
t_un = 0.1 * t_u
training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    t_u = t_un,
    t_c = t_c)

Epoch 500, Loss 7.860115
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957698
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)

In [40]:
params = torch.tensor([1.0, 0.0], requires_grad = True)
learning_rate = 1e-1
optimizer = optim.Adam([params], lr = learning_rate)

In [41]:
training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    t_u = t_u,
    t_c = t_c)

Epoch 500, Loss 7.612900
Epoch 1000, Loss 3.086700
Epoch 1500, Loss 2.928579
Epoch 2000, Loss 2.927644
Epoch 2500, Loss 2.927645
Epoch 3000, Loss 2.927646
Epoch 3500, Loss 2.927645
Epoch 4000, Loss 2.927646
Epoch 4500, Loss 2.927646
Epoch 5000, Loss 2.927645


tensor([  0.5368, -17.3048], requires_grad=True)

In [42]:
def training_loop(n_epochs, optimizer, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
    return params

In [55]:
n_samples = t_u.shape[0]
n_val = int(0.2*n_samples)
shuffled_indices = torch.randperm(n_samples)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
train_indices, val_indices

(tensor([6, 5, 3, 1, 2, 8, 7, 0, 9]), tensor([10,  4]))

In [94]:
train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices] 
val_t_c = t_c[val_indices]

train_t_un = train_t_u*0.1
val_t_un = val_t_u*0.1

In [72]:
def training_loop(n_epochs, optimizer, params, train_t_u, train_t_c, val_t_u, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)
        
        val_t_p = model(val_t_u, *params)
        val_loss = loss_fn(val_t_p, val_t_c)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if epoch % 50 == 0:
            print('Epoch %d, Training Loss %.2f, Validation Loss %.2f' % (epoch, train_loss.float(), val_loss.float()))
    
    return params

In [70]:
params = torch.tensor([1.0, 0.0], requires_grad = True)
learning_rate = 1e-1
optimizer = optim.Adam([params], lr = learning_rate)

In [71]:
training_loop(n_epochs=500, optimizer = optimizer, params = params, 
              train_t_u = train_t_u, train_t_c = train_t_c, val_t_u = val_t_u, val_t_c = val_t_c)

Epoch 50, Training Loss 32.35, Validation Loss 14.88
Epoch 100, Training Loss 26.35, Validation Loss 11.49
Epoch 150, Training Loss 19.50, Validation Loss 9.84
Epoch 200, Training Loss 13.95, Validation Loss 8.28
Epoch 250, Training Loss 9.78, Validation Loss 6.98
Epoch 300, Training Loss 6.88, Validation Loss 5.92
Epoch 350, Training Loss 5.00, Validation Loss 5.08
Epoch 400, Training Loss 3.88, Validation Loss 4.44
Epoch 450, Training Loss 3.27, Validation Loss 3.96
Epoch 500, Training Loss 2.99, Validation Loss 3.60


tensor([  5.1335, -15.9296], requires_grad=True)

In [73]:
def training_loop(n_epochs, optimizer, params, train_t_u, train_t_c, val_t_u, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)
        
        with torch.no_grad():
            val_t_p = model(val_t_u, *params)
            val_loss = loss_fn(val_t_p, val_t_c)
            assert val_loss.requires_grad == False
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if epoch % 50 == 0:
            print('Epoch %d, Training Loss %.2f, Validation Loss %.2f' % (epoch, train_loss.float(), val_loss.float()))
    
    return params

In [81]:
params = torch.tensor([1.0, 0.0], requires_grad = True)
learning_rate = 1e-1
optimizer = optim.Adam([params], lr = learning_rate)

In [75]:
training_loop(n_epochs=500, optimizer = optimizer, params = params, 
              train_t_u = train_t_u, train_t_c = train_t_c, 
              val_t_u = val_t_u, val_t_c = val_t_c)

Epoch 50, Training Loss 27.32, Validation Loss 17.74
Epoch 100, Training Loss 17.60, Validation Loss 12.63
Epoch 150, Training Loss 10.88, Validation Loss 9.16
Epoch 200, Training Loss 6.83, Validation Loss 6.84
Epoch 250, Training Loss 4.66, Validation Loss 5.36
Epoch 300, Training Loss 3.61, Validation Loss 4.46
Epoch 350, Training Loss 3.15, Validation Loss 3.92
Epoch 400, Training Loss 2.97, Validation Loss 3.61
Epoch 450, Training Loss 2.90, Validation Loss 3.43
Epoch 500, Training Loss 2.88, Validation Loss 3.34


tensor([  5.2488, -16.6556], requires_grad=True)

In [85]:
def model(t_u, w, b):
    t_p = w_2*t_u*t_u + w_1*t_u + b
    return t_p

In [86]:
def clalc_forward(t_u, t_c, params,is_train):
    with torch.set_grad_enabled(is_train):
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
    return loss

In [87]:
def training_loop(n_epochs, optimizer, params, train_t_u, train_t_c, val_t_u, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_loss = clalc_forward(train_t_u, train_t_c, params, is_train = True)
        val_loss = clalc_forward(val_t_u, val_t_c, params, is_train = False)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if epoch % 50 == 0:
            print('Epoch %d, Training Loss %.2f, Validation Loss %.2f' % (epoch, train_loss.float(), val_loss.float()))
    
    return params

In [88]:
params = torch.tensor([1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-1
optimizer = optim.Adam([params], lr = learning_rate)

In [89]:
training_loop(n_epochs=500, optimizer = optimizer, params = params, 
              train_t_u = train_t_u, train_t_c = train_t_c, 
              val_t_u = val_t_u, val_t_c = val_t_c)

Epoch 50, Training Loss 7.06, Validation Loss 6.64
Epoch 100, Training Loss 4.39, Validation Loss 3.14
Epoch 150, Training Loss 3.49, Validation Loss 2.44
Epoch 200, Training Loss 3.25, Validation Loss 2.10
Epoch 250, Training Loss 3.18, Validation Loss 1.96
Epoch 300, Training Loss 3.14, Validation Loss 1.91
Epoch 350, Training Loss 3.10, Validation Loss 1.90
Epoch 400, Training Loss 3.06, Validation Loss 1.89
Epoch 450, Training Loss 3.01, Validation Loss 1.89
Epoch 500, Training Loss 2.97, Validation Loss 1.89


tensor([ 0.5050, -0.0422, -4.1968], requires_grad=True)

In [114]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0] #known data
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4] #unknown data
t_c = torch.tensor(t_c).unsqueeze(1)
t_u = torch.tensor(t_u).unsqueeze(1)

In [116]:
linear_model = nn.Linear(1,1)
optimizer = optim.SGD(linear_model.parameters(),lr = 1e-2)